# Single Agent Organ Generator V1 - Updated Workflow

This notebook demonstrates the **Single Agent Organ Generator V1** workflow with the new **Rule Engine** for adaptive requirements capture.

## Key Features

- **Rule Engine**: Adaptive questioning based on what is missing, ambiguous, or would cause rework
- **Three Rule Families**: Completeness (A), Ambiguity (B), Conflict (C)
- **Attempt Strategy**: Infer -> Propose defaults -> Ask targeted questions
- **Rework Cost Ranking**: High-cost questions asked first
- Multi-object workflow with 10-step state machine
- Dynamic question variance tailored to organ type
- Per-object folder structure with versioned artifacts
- 9-section requirements schema

## 1. Setup

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))
from automation import (
    SingleAgentOrganGeneratorV1, WorkflowState, ProjectContext, ObjectContext,
    ObjectRequirements, QUESTION_GROUPS, ORGAN_QUESTION_VARIANTS,
    detect_organ_type, get_tailored_questions, run_single_agent_workflow,
    # Rule Engine components
    RuleEngine, IntentParser, QuestionPlanner,
    RuleFlag, ProposedDefault, RuleEvaluationResult, PlannedQuestion,
    run_rule_based_capture,
)
print("All imports successful!")

## 2. Configuration

In [ ]:
API_KEY = os.environ.get('OPENAI_API_KEY', None)
PROVIDER = 'openai'
MODEL = 'gpt-4'
print(f'Provider: {PROVIDER}, Model: {MODEL}, API Key: {API_KEY is not None}')

## 3. Rule Engine (Adaptive Requirements Capture)

The workflow now uses a **rule engine** instead of fixed question groups. The rule engine adaptively determines what questions to ask based on:

1. **What's missing** (Family A - Completeness)
2. **What's ambiguous** (Family B - Ambiguity)
3. **What would cause expensive rework** (Family C - Conflict)

### 3.1 IntentParser - Extract Values and Detect Ambiguities

In [ ]:
# Test IntentParser with different intents
test_intents = [
    'box 2x6x3 cm with diameter 1mm',
    'dense liver-like network with inlet on the left side',
    'symmetric branching with 2 inlets',
    'perfusion network for tissue engineering',
]

print('IntentParser Analysis:')
print('=' * 70)
for intent in test_intents:
    parser = IntentParser(intent)
    print(f"\nIntent: '{intent}'")
    print(f"  Spatial ambiguity: {parser.has_spatial_ambiguity()}")
    print(f"  Vague quantifiers: {parser.has_vague_quantifiers()}")
    print(f"  Implicit I/O: {parser.has_implicit_io()}")
    print(f"  Symmetry ambiguity: {parser.has_symmetry_ambiguity()}")
    if parser.extracted_values:
        print(f"  Extracted values: {parser.extracted_values}")

### 3.2 RuleEngine - Evaluate Requirements

In [ ]:
# Create empty requirements and evaluate
requirements = ObjectRequirements()
engine = RuleEngine(organ_type='liver')

result = engine.evaluate(requirements, intent='dense liver network')

print('RuleEngine Evaluation:')
print('=' * 70)
print(f"Generation ready: {result.is_generation_ready}")
print(f"\nMissing fields ({len(result.missing_fields)}):")
for flag in result.missing_fields[:3]:
    print(f"  [{flag.severity}] {flag.message}")
print(f"\nAmbiguity flags ({len(result.ambiguity_flags)}):")
for flag in result.ambiguity_flags:
    print(f"  [{flag.severity}] {flag.message}")
print(f"\nProposed defaults ({len(result.proposed_defaults)}):")
for prop in result.proposed_defaults[:3]:
    print(f"  {prop.field}: {prop.value} ({prop.reason})")

### 3.3 QuestionPlanner - Plan Minimal Questions

In [ ]:
# Plan questions based on evaluation result
planner = QuestionPlanner()
questions = planner.plan(result, max_questions=4)

print('Planned Questions (ranked by rework cost):')
print('=' * 70)
for i, q in enumerate(questions, 1):
    default_str = f' [{q.default_value}]' if q.default_value else ''
    print(f"{i}. [{q.rework_cost.upper()}] {q.question_text}{default_str}")
    print(f"   Field: {q.field}")
    print(f"   Reason: {q.reason}")
    print()

### 3.4 Attempt Strategy Comparison

The rule engine uses a 3-tier attempt strategy. More explicit intents result in fewer questions:

In [ ]:
# Demonstrate how explicit values reduce questions
explicit_intent = 'box 2x6x3 cm, diameter 1mm, 2 inlets, 1 outlet'
vague_intent = 'dense branching network on the left side'

for intent in [explicit_intent, vague_intent]:
    parser = IntentParser(intent)
    req = ObjectRequirements()
    eng = RuleEngine()
    res = eng.evaluate(req, intent)
    qs = planner.plan(res, max_questions=4)
    
    print(f"\nIntent: '{intent}'")
    print(f"  Extracted values: {len(parser.extracted_values)}")
    print(f"  Ambiguities detected: {len(res.ambiguity_flags)}")
    print(f"  Questions needed: {len(qs)}")

## 4. Dynamic Question Variance (New Feature)

The workflow detects organ type from your description and tailors questions for Groups C, D, E.

### 4.1 Organ Type Detection

In [ ]:
test_intents = [
    'liver vascular network',
    'kidney with renal arteries',
    'lung pulmonary tree',
    'coronary arteries for heart',
    'generic tubular structure',
]
print('Organ Detection Results:')
print('-' * 50)
for intent in test_intents:
    organ = detect_organ_type(intent)
    print(f"{organ:8} <- {intent}")

### 4.2 Tailored Questions Comparison

In [ ]:
print('Group C (Inlets/Outlets) - First Question by Organ Type:')
print('=' * 60)
for organ in ['liver', 'kidney', 'lung', 'heart', 'generic']:
    q = get_tailored_questions(f'{organ} network')
    group_c = q['C']
    first_q = group_c['questions'][0]
    print(f"\n{organ.upper()} ({group_c['name']}):\n  Q: {first_q[1]}\n  Default: {first_q[2]}")

## 5. Workflow States

In [ ]:
print('Workflow States:')
for i, state in enumerate(WorkflowState, 1):
    print(f"{i:2}. {state.value}")

## 6. Question Groups (A-G)

In [ ]:
print('Question Groups:')
for key, group in QUESTION_GROUPS.items():
    print(f"Group {key}: {group['name']} ({len(group['questions'])} questions)")

## 7. Organ-Specific Question Variants

In [ ]:
print('Organ-Specific Variants:')
for organ, variants in ORGAN_QUESTION_VARIANTS.items():
    print(f"\n{organ.upper()}: {variants['description']}")
    for g in ["C", "D", "E"]:
        if g in variants:
            print(f"  Group {g}: {variants[g]['name']}")

## 7. Running the Workflow

In [ ]:
# Quick start (uncomment to run):
# context = run_single_agent_workflow(provider=PROVIDER, model=MODEL, api_key=API_KEY)
print('Uncomment above to run interactive workflow')

## 8. Per-Object Folder Structure

In [ ]:
print("""outputs/<project>/
├── project.json
└── objects/<object>/
    ├── 00_intent/     (intent.txt, requirements_v001.json)
    ├── 01_spec/       (spec_v001.json)
    ├── 02_code/       (generate_v001.py)
    ├── 03_outputs/    (network_v001.json)
    ├── 04_mesh/       (mesh_v001_network.stl)
    ├── 05_analysis/   (analysis_v001.json)
    ├── 06_validation/ (validation_v001.json)
    ├── 07_iterations/
    └── 08_final/      (void.stl, manifest.json)""")

## 9. Requirements Schema (9 Sections)

In [ ]:
sections = [
    ("1. Identity", "object_name, slug, version"),
    ("2. Frame of Reference", "origin, axes, viewpoint, units"),
    ("3. Domain", "type, size, center, margin"),
    ("4. Inlets/Outlets", "positions, radii, directions"),
    ("5. Topology Intent", "style, target_terminals, max_depth"),
    ("6. Geometry Intent", "segment_length, tortuosity, branch_angle"),
    ("7. Constraints", "min_radius, min_clearance, boundary_buffer"),
    ("8. Embedding & Export", "domain, voxel_pitch, stl_units"),
    ("9. Acceptance Criteria", "min_radius, terminals_range, watertight"),
]
print('Requirements Schema:')
for section, fields in sections:
    print(f"{section}: {fields}")

## 10. Summary

This notebook demonstrated:
1. Dynamic Question Variance for liver, kidney, lung, heart, generic
2. Organ Detection from user intent
3. 10-Step Workflow states
4. 7 Question Groups (A-G)
5. 9-Section Requirements Schema
6. Per-Object Folder Structure